## Demo: Validating a Cerebellar Purkinje Cell Model Against Experimental Data

In this demo, you will download a published NEURON model of a Purkinje cell, run several different simulation experiments, and compare the results against experimental data.

The demo has several sections:
1. Install software
2. Download the model
3. Check if _'Model Catalog'_ and _'Validation Framework'_ apps exist in Collab
4. Get information about the model from the HBP Model Catalog
5. View available tests and select one of the tests
6. Download and run the validation test
7. Inspect the result of the validation test

To run the demo, execute each of the notebook cells in order.

### 1. Install software
(no user intervention required)

In [ ]:
%matplotlib inline
from IPython.display import Markdown as markdown

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version

req_packages = {    
                    "hbp_service_client"       : {"min_version": "1.1.0",  "install_version": "1.1.0"},
                    "sciunit"                  : {"min_version": "",       "install_version": "git://github.com/scidash/sciunit.git@9e61c060cd8e5c6d7bce675d37c398987a9491c5"},
                    "hbp_validation_framework" : {"min_version": "0.5.1",  "install_version": "0.5.1"},
                    "numpy"                    : {"min_version": "1.16.2", "install_version": "1.16.2"},    
                    "fpdf"                     : {"min_version": "1.7.2",  "install_version": "1.7.2"},
                    "PyPDF2"                   : {"min_version": "1.26.0", "install_version": "1.26.0"},
                    "neo"                      : {"min_version": "0.5.2",  "install_version": "0.5.2"},
                    "elephant"                 : {"min_version": "0.4.3",  "install_version": "0.4.3"}
               }

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg_name, pkg_vinfo in req_packages.items():
        print("Checking for package: {}".format(pkg_name))        
        try:
            pkg_resources.get_distribution(pkg_name)
            if pkg_name == "sciunit":
                !pip uninstall -y sciunit               
            current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
            if not pkg_vinfo["min_version"] or current_version < parse_version(pkg_vinfo["min_version"]) or current_version > parse_version(pkg_vinfo["install_version"]):
                    print("\t{}: current version = {}".format(pkg_name, current_version))
                    print("\tAnother version of {} will be installed.".format(pkg_name))
                    raise
        except:            
            if "github.com" in pkg_vinfo["install_version"]:
                os.system("pip install --quiet --no-cache-dir --upgrade git+{}".format(pkg_vinfo["install_version"]))
            else:
                os.system("pip install --quiet --no-cache-dir --upgrade {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                
            if pkg_name == "hbp_service_client":
                from IPython.display import HTML
                display(HTML('''<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
                Jupyter.notebook.dirty = false; window.location.reload(); })</script>'''))
                
install_req_packages()                 

import sciunit
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

In [ ]:
!pip freeze

In [ ]:
%%capture
!pip install -U neo elephant 
# !pip install -U git+https://github.com/lungsi/cerebellum-unit
# !pip install -U git+https://github.com/apdavison/cerebellum-unit.git@dev
# !pip uninstall -y cerebunit
!pip install -U --no-cache-dir git+https://github.com/appukuttan-shailesh/cerebellum-unit.git@usecase_mods

### 2. Download the model

The model is stored in a repository on Github. Here we will use Git to get a copy of the repository.

In [ ]:
%%bash
if [ ! -d "hbp-cerebellum-models" ] ; then
    # if this is the first time running this notebook, clone the Git repository
    #git clone https://github.com/lungsi/hbp-cerebellum-models.git
    git clone https://github.com/appukuttan-shailesh/hbp-cerebellum-models
    cd hbp-cerebellum-models
    git fetch
    git checkout add-id
else
    # otherwise pull any recent changes
    cd hbp-cerebellum-models
    git pull
fi

In [ ]:
cd hbp-cerebellum-models

### Create an instance of the Purkinje cell model

In [ ]:
from models import cells
pc = cells.PC2015Masoli.PurkinjeCell()

### Run sample simulations
This part is not necessary for validating the model, but let's run a simulation to get an idea of how the model behaves:

#### ... without current injection (model fires spontaneously)

In [ ]:
pc.set_simulation_properties({"dt": 0.025,
                             "celsius": 37,
                             "v_init": -60,
                             "tstop": 200})
pc.produce_voltage_response()  # this runs the simulation, it may take a couple of minutes

In [ ]:
from models import plot_manager as pm
pm.visualize_voltages(model_name="PC2015Masoli", region_of_interest="vm_soma")

#### ... with external current injection

In [ ]:
pc.set_simulation_properties({"dt": 0.025,
                             "celsius": 37,
                             "v_init": -60,
                             "tstop": 200})
# set current injection stimulus
stimuli = pc.set_stimulation_properties({"current1": {"amp": 0.5, "dur": 100, "delay": 50}})
pc.produce_voltage_response()  # this runs the simulation, it may take a couple of minutes

# remove the current injection to avoid interference with subsequent simulations
stimuli = pc.set_stimulation_properties({})

In [ ]:
pm.visualize_voltages(model_name="PC2015Masoli", region_of_interest="vm_soma")

### 3. Check if 'Model Catalog' and 'Validation Framework' apps exist in Collab
The HBP Model Catalog contains information about models of many kinds, from detailed biophysical models to abstract conceptual models, from models of sub-cellular mechanisms to models of entire brain regions. The HBP Validation Service lets us keep track of all the validation tests that have been run for these different models.

Just as the Model Catalog holds information about models, the Validation Test Library holds information about validation tests. You can access the validation test library using a graphical app, or using the Python client. For more information, see the [documentation](http://hbp-validation-client.readthedocs.io/).

In [ ]:
test_library = TestLibrary()
catalog = ModelCatalog.from_existing(test_library)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = catalog.exists_in_collab_else_create(collab_id)
catalog.set_app_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = test_library.exists_in_collab_else_create(collab_id)
test_library.set_app_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

print("\n\nLink to Model Catalog app:")
print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}".format(str(collab_id),str(MCapp_navID)))
print("\nLink to Test Library app:")
print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}".format(str(collab_id),str(VFapp_navID)))

### 4. Get information about the model from the HBP Model Catalog

In [ ]:
from hbp_validation_framework import ModelCatalog
catalog = ModelCatalog()

A model should have an attribute `model_uuid` that uniquely identifies it in the Model Catalog. The Purkinje cell model we're validating here is already registered in the catalog. 
If you want to upload the validation test results for your own model to the Validation service,
you will first have to register your model using the Model Catalog app (see left-hand menu, or link above), or using the Python client. 

A model may contain multiple model instances. Each instance defines a particular version (identified by the attribute `model_version`) of a model by specifying the location of the source code for the model. A model may have multiple versions (model instances) which could vary, for example, in values of their biophysical parameters. Improvements and updates to a model would be considered as different versions (instances) of that particular model.

For more information, see the [documentation](http://hbp-validation-client.readthedocs.io/).

In [ ]:
# model_uuid has been specified in model source code; hence commented out here
# pc.model_uuid = "22dc8fd3-c62b-4e07-9e47-f5829e038d6d"

# sciunit provides models the ability to determines its version from Git
pc.model_version = pc.version

In [ ]:
model_info = catalog.get_model(pc.model_uuid)
markdown(model_info["description"])

### 5. Find validation tests for Purkinje cell models

In [ ]:
from hbp_validation_framework import TestLibrary
test_library = TestLibrary()

In [ ]:
tests = test_library.list_tests(cell_type="Purkinje Cell")

In [ ]:
for test in tests:
    print("{} ({})".format(test["name"], test["alias"]))
print("Note: the list above is in the form `test_name (test_alias)`")

__Note:__ The `alias` attribute of a test is an (optional) unique human-readable identifier that can be assigned to a test. This makes it easier to refer to a test, rather than using the `test_uuid`. The latter is mandatory for every test and can be used to specify/identify tests.

### Load the test of interest

Validation tests are implemented as Python classes. To load the test, we need to choose the version we want. First, we can get a list of versions:

In [ ]:
test_library.list_test_instances(alias="PCSpontaneousFiringTest")
# test_library.list_test_instances(alias="PCComplexBurstingTest")

Now we can get an instance of the Python class that implements the test, initialized with the experimental reference data. 

In [ ]:
test = test_library.get_validation_test(alias="PCSpontaneousFiringTest", version="0.1.dev")
# test = test_library.get_validation_test(alias="PCComplexBurstingTest", version="0.1.dev")

In [ ]:
print(test.description)

### Look at the test data

In [ ]:
test.observation

### 6. Run the validation test

Now we can run the test on the Purkinje neuron model:

In [ ]:
score = test.judge(pc)

In [ ]:
score.score

But what does this score mean? Is it a good or bad result? To find out, we can check the score's description:

In [ ]:
score.description

In [ ]:
print score._description

We can remind ourselves of the experimental data (the "observation") we are validating against:

In [ ]:
score.observation

and see more detail of the simulation results (the "prediction")

In [ ]:
score.prediction

### Save the validation result to the result database

Below we first extract the `collab_id`. Every test result in the Validation Framework is registered with a `project` attribute. This corresponds to the ID of the Collab in which the test was executed. The _Storage_ of this Collab would be utilized for storing and test related files.

In [ ]:
def get_collab_id():
    """Workaround to get the collab id from within a notebook"""
    import inspect
    import re
    
    somecode = inspect.getsource(get_collab_storage_path)
    match = re.search(r'collab_id=(?P<collab_id>\d+)', somecode)
    return match.groupdict()['collab_id']

In [ ]:
collab_id = get_collab_id()

In [ ]:
from hbp_validation_framework.datastores import CollabDataStore
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
folder_name = "results_{}_{}_{}".format(pc.name, pc.model_uuid[:8], timestamp)
collab_storage = CollabDataStore(base_folder=folder_name)

In [ ]:
response = test_library.register_result(score, data_store=collab_storage, project=collab_id)

### 7. Browse the result database

In the left-hand menu, you should see a menu item "Validation Framework" (if not, please refresh the page so that the newly added apps are visible).
Right-click on this to open in a separate tab. 
You will now see a list of all registered models and validation tests.
In the "Select cell type" dropdown, choose "Purkinje cell" to only show models of this cell type
and tests for such models.

Now click on "PCSpontaneousFiringTest".
This shows a description of the test. 
You can click on "Version" to find the code for the test, 
on "Results" to see the scores for each time this test has been run,
and on "Comments" to see a discussion about this test and the associated data.


In [ ]:
print("Alternatively, you can click here to directly go the result that was currently registered:")
print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), response))

## Summary

The steps shown in this notebook can be applied to any model and any test.
To recap, the general steps are as follows:

- register your model, using the Model Catalog app
- on the computer where you will run the tests (or in a Collab notebook), install your model code, the test code, and the `hbp_validation_framework` Python package.
- using the `TestLibrary` client, download and instantiate a test.
- call `test.judge(model)`
- again using the `TestLibrary` client, upload the test results to the Validation service.